In [2]:
from Seva import Seva
# import mysql.connector
obj = Seva()

In [5]:
obj.getAllBerita([], 1, 'tips-n-rekomendasi', 'properti')

page  1
https://www.seva.id/properti/blog/category/tips-n-rekomendasi/page/1


ValueError: invalid literal for int() with base 10: 'https://www.seva.id/properti/?p=271'